In [1]:
# import getpass
import os
import bs4
from os import path
os.environ['USER_AGENT'] = 'myagent'
from langchain import hub
from langchain_ollama import ChatOllama
from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.document_loaders import PyMuPDFLoader
# from langchain_unstructured import UnstructuredLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict

# os.environ["LANGSMITH_TRACING"] = "false"

In [2]:
embeddings = OllamaEmbeddings(model="deepseek-r1:70b")
vector_store = Chroma(
    embedding_function=embeddings,
    persist_directory="./chroma_db/earthquake/",
)

In [15]:
# Load and chunk contents of the blog
#loader = WebBaseLoader(
#    web_paths=("http://www.dengzhou.gov.cn/dzszj/zjxw/zwyw/webinfo/2024/11/1723956574402713.htm",),
#    bs_kwargs=dict(
#        parse_only=bs4.SoupStrainer(
#            class_=("post-content", "post-title", "post-header")
#        )
#    ),
#)
libdir = "../library/Earthquake"
papers = []
for f in os.listdir(libdir):
    if f.endswith(".pdf"):
        papers.append(path.normpath(path.abspath(path.join(libdir, f))))
docs = []
for f in papers:
    loader = PyMuPDFLoader(
        f,
        mode = 'single'
)
    docs += loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=4096, chunk_overlap=256)
all_splits = text_splitter.split_documents(docs)

In [ ]:
# Index chunks
_ = vector_store.add_documents(documents=all_splits)

In [4]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

llm = ChatOllama(
    model="deepseek-r1:70b",
    temperature=0.0,
    num_predict=-1
)

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}

# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [6]:
response = graph.invoke({"question": "介绍地震过程的物理模型、数值仿真与数据分析。"})
print(response["answer"])

<think>
好的，我现在需要回答关于地震过程的物理模型、数值仿真与数据分析的问题。首先，根据提供的上下文，物理模型部分提到了热-机械耦合，这涉及到温度对滑动行为的影响，以及粘性强度和摩擦发热之间的相互作用。这可能导致变形局部化和剪切不稳定性，从而解释深层地震的发生机制。

接下来是数值仿真部分，上下文中提到了通过阴影焦散法来分析材料在加载时的应力状态变化，这种方法可以检测到强变形区的出现，进而预测断裂的位置。这种实验室技术可能被引入地震研究，以便更好地理解断层滑动和破裂过程。

最后是数据分析部分，上下文提到了通过定量计算断裂力学参数，如应力变化和材料特性变化，这有助于精确预测地震发生的条件和地点。这些方法结合起来，可以为研究地震提供物理模型、仿真工具和数据支持，从而深入理解地震机制。
</think>

地震过程的物理模型主要基于热-机械耦合，探讨温度对滑动行为的影响及粘性强度与摩擦发热的相互作用。数值仿真通过阴影焦散法分析材料应力变化，检测变形区以预测断裂位置。数据分析则定量计算断裂力学参数，研究地震发生机制。


In [35]:
import langchain_ollama
llm2 = langchain_ollama.ChatOllama(model='llama3')

In [28]:
llm.get_num_tokens("hello world")

OSError: Can't load tokenizer for 'gpt2'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'gpt2' is the correct path to a directory containing all relevant files for a GPT2TokenizerFast tokenizer.